In [1]:
"""
This is a template algorithm on Quantopian for you to adapt and fill in.
"""
import quantopian.algorithm as algo
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.domain import US_EQUITIES
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.data.morningstar import Fundamentals
import numpy as np
import pandas as pd

#function called once at beginning 
#required for implementation 
def initialize(context):
    """
    Called once at the start of the algorithm.
    """
    # bring in data for NASDAQ
    context.stock=sid(27026)
    
    # use SPY as a benchmark
    set_benchmark(sid(8554))
    
    # Rebalance every day, 1 hour after market open.
    algo.schedule_function(
        rebalance,
        algo.date_rules.every_day(),
        algo.time_rules.market_open(hours=1),
    )

    # Record tracking variables at the end of each day.
    algo.schedule_function(
        record_vars,
        algo.date_rules.every_day(),
        algo.time_rules.market_close(),
    )

    # Create our dynamic stock selector.
    algo.attach_pipeline(make_pipeline(), 'pipeline')

def make_pipeline():
    """
    A function to create our dynamic stock selector (pipeline). Documentation
    on pipeline can be found here:
    https://www.quantopian.com/help#pipeline-title
    """

    # Base universe set to the QTradableStocksUS
    base_universe = QTradableStocksUS()
    
    #get prices for companies on nasdaq 100
    exchange = morningstar.share_class_reference.exchange_id.latest
    my_exchange = exchange.element_of(["NAS"])
    market_cap = Fundamentals.market_cap.latest
    nasdaq_100 = market_cap.top(103, mask = my_exchange)
    company_name = morningstar.company_reference.short_name.latest
    
    # Factor of yesterday's close price.
    price = USEquityPricing.close.latest
    
    # 1-month (21 trading day) moving average factor
    fast_ma = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=21, mask=my_exchange)
    
    # 6-month (126 trading day) moving average factor
    slow_ma = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=126, mask=my_exchange)
    
    momentum = fast_ma/slow_ma
    momentum_factor = momentum.zscore()
    
    
    pipe = Pipeline(
        columns={
            'company_name': company_name,
            'market_cap' : market_cap,
            'price' : price,
            'momentum_factor': momentum_factor,
        },
        screen=nasdaq_100 & my_exchange,
    )
    return pipe

#Optional not necessary

def before_trading_start(context, data):
    """
    Called every day before market open.
    """
    context.output = algo.pipeline_output('pipeline')

    # These are the securities that we are interested in trading each day.
    context.security_list = context.output.index

    
def calculate_zscore(context, data):
    """
    Calculates the zscore of percent daily returns over the past trading month
    """
    # use the past 21 days
    stock_price = data.history(context.stock, 'close', 21, '1d')
    
    # calculate percent change between each day
    pct_change = stock_price.pct_change()
    
    # calculate average percent change
    stock_mean = np.mean(pct_change)
 
    # calculate standard deviation of percent change
    stock_stdev = np.std(pct_change)
    
    # calculate zscore using most recent percent change
    zscore = (pct_change[-1] - stock_mean)/stock_stdev
    
    #save this variable for plotting
    record(pct_change=pct_change[-1])
    return zscore


def rebalance(context, data):
    """
    Execute orders according to our schedule_function() timing.
    """
    stock = context.stock
    stock_data = data.current(stock, 'price')
    # set window length for 6-month (126 days) and 1-month (21 days) moving averages
    long_window = history(126,'1d','price')
    short_window = history(21,'1d','price')
    
    # calculate moving averages
    long_ma = long_window[context.stock].mean()
    short_ma  = short_window[context.stock].mean()
    
    # calculate the zscore of daily percent change
    zscore = calculate_zscore(context, data)
    
    # decision making for purchasing/selling stocks
    # when to buy
    if short_ma > long_ma or zscore > 1.5:
        order_target_percent(sid(27026), 1) #buy all
        
    # when to sell
    # if 21day avg falls below 126day avg OR stock price's percent change is
    # more than than 1.5 standard deviations below average
    elif short_ma < long_ma or zscore < -1.5:
        order_target(sid(27026), 0) #sell all
        
    record(z_score = zscore,
          short_avg = short_ma,
          long_avg = long_ma,
          price = stock_data)

#function 
def record_vars(context, data):
    """
    Plot variables at the end of each day.
    """


#called at the end of each market minute 
def handle_data(context, data):
    """
    Called every minute.
    """

ImportError: No module named 'quantopian.algorithm'